In [15]:
from dataset import *

data_files = ['D:/senior/aiCourse/dataSource/comment_classification/output/train.json']
vocab_file = 'D:/senior/aiCourse/dataSource/comment_classification/output/vocab.txt'
label_file = 'D:/senior/aiCourse/dataSource/comment_classification/labels.txt'
enb_file = 'D:/senior/aiCourse/dataSource/comment_classification/embedding/embedding.txt'
batch_size = 128
reverse = False
split_word = True
max_len = 400


In [16]:
data_files=['../../../dataSource/comment_classification/output/train.json']
vocab_file = '../../../dataSource/comment_classification/output/vocab.txt'
label_file = '../../../dataSource/comment_classification/labels.txt'
enb_file = '../../../dataSource/comment_classification/output/embedding.txt'

In [17]:
dataset1 = DataSet(  data_files,  vocab_file,   label_file,   batch_size, reverse=  reverse, split_word=  split_word, max_len=  max_len)

# vocab size:  50000
# vocab size:  20
# Start to preprocessing data...
# load data from ../../../dataSource/comment_classification/output/train.json ...
# Got 105000 data items with 820 batches


In [18]:
for i,(source, lengths, targets, _) in enumerate(dataset1.get_next()):
    print(len(source[9]))
    break

402


In [19]:
def dig_lists(l):
    output = []
    for e in l:
        if isinstance(e, list):
            output += dig_lists(e)
        else:
            output.append(e)
    return(output)

In [20]:
def pad_sequences(comment_to_id,maxlen,padding,truncating):
    features = np.zeros((len(comment_to_id), maxlen), dtype=int)
    for i,comment in enumerate(comment_to_id):
        if len(comment) <= maxlen and padding == 'pre':
            features[i, -len(comment):] = np.array(comment)[:maxlen]
        if len(comment) <= maxlen and padding == 'post':
            features[i, :len(comment)] = np.array(comment)[:maxlen]
        if len(comment) > maxlen and truncating == 'post':
            features[i, :] = np.array(comment)[:maxlen]
        if len(comment) > maxlen and truncating == 'pre':
            features[i, :] = np.array(comment)[len(comment)-maxlen:]           
    return features

def split_dataset(pad_comments,labels,split_frac):
    split_index = int(len(pad_comments)*split_frac)
    data_list = list(zip(pad_comments, labels))
    random.shuffle(data_list)
    pad_comments, labels = zip(*data_list)
    x_train, x_test = pad_comments[:split_index], pad_comments[split_index:]
    y_train, y_test = labels[:split_index], labels[split_index:]
    return x_train,y_train,x_test,y_test 

In [21]:
comment_to_id = [x[0] for x in dataset1._raw_data]
pad_comments = pad_sequences(comment_to_id,maxlen=max_len,padding='post',truncating='post')

In [22]:
labels = [x[2].flatten().tolist() for x in dataset1._raw_data]

In [23]:
import random
x_train,y_train,x_test,y_test = split_dataset(pad_comments[:10000],labels,0.8)

# Model

In [24]:
import pandas as pd
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.engine.topology import Layer
from keras import initializers as initializers, regularizers, constraints
from keras.callbacks import Callback
from keras.layers import concatenate,GlobalMaxPooling1D,GlobalAveragePooling1D,SpatialDropout1D,Embedding, Input, Dense, LSTM, GRU, Bidirectional, TimeDistributed
from keras import backend as K
from keras.models import Model
import keras.layers as layers
from sklearn.metrics import roc_auc_score
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense,Dropout,Embedding,LSTM,Bidirectional,Layer
from keras.datasets import imdb


In [40]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test) 

In [70]:
def get_model():
    embed_size = 256
    inp = Input(shape=(400, ))
    x = Embedding(input_dim=len(dataset1.w2i), output_dim=embed_size,)(inp)
    print(x)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(GRU(120, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(80, activation="relu")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [67]:
len(dataset1.w2i)

50000

In [78]:
def get_model():
    embed_size = 256
    model = Sequential()   
    
    model.add( Embedding(50000, embed_size,input_length=400))
    
    
    model.add(Bidirectional(GRU(100, return_sequences=True)))
    model.add(SpatialDropout1D(0.2))
              
    #model.add(Bidirectional(GRU(100, return_sequences=True)))
    
    model.add(GlobalAveragePooling1D())
    model.add(Dense(80, activation="relu"))
    
    
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [79]:
model = get_model()

In [80]:
print(model.summary())
history = model.fit(x_train, y_train,epochs=5,verbose=1, batch_size=batch_size)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 400, 256)          12800000  
_________________________________________________________________
bidirectional_20 (Bidirectio (None, 400, 200)          214200    
_________________________________________________________________
spatial_dropout1d_11 (Spatia (None, 400, 200)          0         
_________________________________________________________________
global_average_pooling1d_7 ( (None, 200)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 80)                16080     
Total params: 13,030,280
Trainable params: 13,030,280
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
1152/8000 [===>..........................] - ETA: 4:57 - loss: 1.3092 - acc: 0.7500

KeyboardInterrupt: 